In [0]:
%pip install xgboost

In [0]:
print ("cambio1")

In [0]:
print ("cambio2")

In [0]:

# Importamos las librerías necesarias para análisis de datos, visualización y modelado
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score, f1_score, classification_report
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns


In [0]:

# Cargamos el dataset desde el archivo CSV
# Este dataset contiene información de clientes de telecomunicaciones y si abandonaron el servicio (churn)
df = pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv")
# Mostramos las primeras filas para inspección inicial
df.head()


In [0]:

# Mostramos información general del dataset: tipos de datos, nulos, etc.
df.info()

# Mostramos estadísticas descriptivas de todas las columnas
df.describe(include='all')


In [0]:

# Visualizamos la distribución de la variable objetivo 'Churn'
sns.countplot(data=df, x='Churn')
plt.title('Distribución de la variable Churn')
plt.show()


In [0]:

# Convertimos TotalCharges a numérico, ya que puede contener valores vacíos
# Imputamos los valores nulos con la mediana
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df['TotalCharges'].fillna(df['TotalCharges'].median(), inplace=True)

# Calculamos la matriz de correlación entre variables numéricas relevantes
corr = df[['tenure', 'MonthlyCharges', 'TotalCharges']].corr()

# Visualizamos la matriz de correlación con un heatmap
sns.heatmap(corr, annot=True, cmap='coolwarm')
plt.title('Matriz de correlación')
plt.show()


In [0]:

# Visualizamos cómo varían los cargos mensuales según el tipo de contrato
sns.boxplot(data=df, x='Contract', y='MonthlyCharges')
plt.title('Monthly Charges por tipo de contrato')
plt.xticks(rotation=45)
plt.show()


In [0]:

# Visualizamos la cantidad de clientes por tipo de servicio de internet y si hicieron churn
sns.countplot(data=df, x='InternetService', hue='Churn')
plt.title('Distribución de clientes por tipo de Internet y Churn')
plt.xticks(rotation=45)
plt.show()


In [0]:

# Visualizamos la relación entre tiempo de permanencia (tenure) y cargos totales
# Segmentamos por churn para ver diferencias
sns.scatterplot(data=df, x='tenure', y='TotalCharges', hue='Churn')
plt.title('Relación entre Tenure y TotalCharges por Churn')
plt.show()


In [0]:

# Eliminamos la columna customerID por no aportar valor predictivo
df.drop("customerID", axis=1, inplace=True)

# Codificamos la variable objetivo Churn como 0 (No) y 1 (Yes)
df["Churn"] = df["Churn"].map({"Yes": 1, "No": 0})

# Codificamos todas las variables categóricas con LabelEncoder
cat_cols = df.select_dtypes(include=['object']).columns
for col in cat_cols:
    df[col] = LabelEncoder().fit_transform(df[col])


In [0]:

# Separamos las variables independientes (X) y la dependiente (y)
X = df.drop("Churn", axis=1)
y = df["Churn"]

# Dividimos el dataset en entrenamiento (70%) y prueba (30%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [0]:

# Creamos y entrenamos el modelo XGBoost para clasificación
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)


In [0]:

# Realizamos predicciones sobre el conjunto de prueba
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]  # Probabilidad de clase positiva

# Calculamos métricas de evaluación
roc_auc = roc_auc_score(y_test, y_proba)
f1 = f1_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

# Mostramos los resultados
print("ROC-AUC Score:", roc_auc)
print("F1 Score:", f1)
print("Classification Report:", report)


In [0]:

# Extraemos la importancia de cada variable en el modelo
importancia = model.feature_importances_
features = X.columns

# Visualizamos la importancia con un gráfico de barras
plt.figure(figsize=(12,6))
sns.barplot(x=importancia, y=features)
plt.title("Importancia de Variables - Modelo XGBoost")
plt.xlabel("Importancia")
plt.ylabel("Características")
plt.tight_layout()
plt.show()
